<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3B: Data_Cleaning

### Contents:
- [Import Libraries](#Import-Libraries)
- [Import Raw Scrapped Data](#Import-Raw-Scrapped-Data)
- [Useful Data Summary](#Useful-Data-Summary)
- [Initial Dataframe Filtering/Cleaning](#Initial-Dataframe-Filtering/Cleaning)
- [Text Processing](#Text-Processing)
- [Save Cleaned Data into CSV](#Save-Cleaned-Data-into-CSV)

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import nltk
import regex as re

from bs4 import BeautifulSoup

#used to find most frequent words in the texts
from collections import Counter

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

#hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)

## Import Raw Scrapped Data

In [3]:
amd = pd.read_csv('../data/amd_raw.csv')

In [4]:
nvidia = pd.read_csv('../data/nvidia_raw.csv')

## Useful Data Summary

From the analysis of the sample data scraped in part 3A, these were the column that were identified as useful. The dataframes will be filtered and cleaned accordingly.

<b>Data to keep</b>:
- subreddit --> target y
- title
- selftext
- media_only (will use to ensure none of our posts collected have only media in the contents, to drop afterwards)

<b>Data to keep but may drop afterwards (numerical, only use if correlation with target can be proven)</b>:
- num_comments
- num_crossposts
- score
- total_awards_received

<b>Initial cleaning to be done</b>:
- drop all duplicates, ensure only unique posts in dataframe
- using media_only column to remove that consists of only media
- title + selftext --> interaction term the post title and post body
- drop rows with NULL values in text column
- change subreddit column values to numerical format (amd = 1, nvidia = 0)
- keep 10 000 posts per subreddit to ensure balanced datasets for models to train and test on

## Initial Dataframe Filtering/Cleaning

In [5]:
#check for null values
print('AMD NULL value check:\n', amd.isnull().sum())
print('\nNVIDIA NULL value check:\n', nvidia.isnull().sum())

AMD NULL value check:
 subreddit                   0
title                       0
selftext                 3699
media_only                  0
num_comments                0
num_crossposts              0
score                       0
total_awards_received       0
dtype: int64

NVIDIA NULL value check:
 subreddit                   0
title                       0
selftext                 3483
media_only                  0
num_comments                0
num_crossposts              0
score                       0
total_awards_received       0
dtype: int64


The 'selftext' column seems to have NULL values along with cells with '[removed]' values. We will clean these up to prepare the 'selfteext' column to create an iteraction term with the 'title' column.

In [6]:
#check datatypes of columns
print('AMD datatypes:\n', amd.dtypes)

print('\nNVIDIA datatypes:\n', nvidia.dtypes)

AMD datatypes:
 subreddit                object
title                    object
selftext                 object
media_only                 bool
num_comments              int64
num_crossposts            int64
score                     int64
total_awards_received     int64
dtype: object

NVIDIA datatypes:
 subreddit                object
title                    object
selftext                 object
media_only                 bool
num_comments              int64
num_crossposts            int64
score                     int64
total_awards_received     int64
dtype: object


In [7]:
#number of columns and rows before dropping duplicated posts
print(amd.shape)
print(nvidia.shape)

(12000, 8)
(12000, 8)


In [8]:
#drop duplicated posts
amd = amd[amd['title'].duplicated() == False]
nvidia = nvidia[nvidia['title'].duplicated() == False]

In [9]:
#number of columns and rows after dropping duplicated posts
print(amd.shape)
print(nvidia.shape)

(11604, 8)
(11757, 8)


In [10]:
#drop posts that contains only media
amd = amd[amd['media_only'] == False]
nvidia = nvidia[nvidia['media_only'] == False]

In [11]:
print(amd.shape)
print(nvidia.shape)

(11604, 8)
(11757, 8)


In [12]:
#drop media_only column
amd = amd.drop(columns='media_only')
nvidia = nvidia.drop(columns='media_only')

In [13]:
amd.head()

,subreddit,title,selftext,num_comments,num_crossposts,score,total_awards_received
0,Amd,3700x vs 5600x,[removed],0,0,1,0
1,Amd,How much is the difference between Amd 7 5800 ...,[removed],0,0,1,0
2,Amd,RX 6800 vs RX 6700 XT,[removed],0,0,1,0
3,Amd,Will an AMD Wraith Stealth cooler fit on an AM...,[removed],1,0,1,0
4,Amd,What does 1.4v do to a 3600xt? Let's find out.,[removed],0,0,1,0


For the empty cells (NA values) in the column 'selftext', we will clean it up and fill it in with '' instead of imputing them since imputing may introduce additional duplicates in the model.

In [14]:
#check if 'selftext' column has empty cells
if amd['selftext'].isnull().sum() != 0:
    #if there is null values, fill those cells with '' to prepare for interaction term
    amd['selftext'] = amd['selftext'].fillna('')
    
if nvidia['selftext'].isnull().sum() != 0:
    #if there is null values, fill those cells with '' to prepare for interaction term
    nvidia['selftext'] = nvidia['selftext'].fillna('')
    
print(np.unique(amd['selftext']), '\n')
print(np.unique(nvidia['selftext']))


[''
 "\n\n \n\nbasically, when I download the drivers just before finishes installation it goes black screen and then i have to put the computer into sleep mode then restart. Then it says its finishes installation then starts flickering then i need to restart properly. After that, it is fine unless the computer then sleeps after not being on for some time ( its a gaming laptop) and when usually a spacebar tap should turn it on it goes black screen again. Also when i change refresh rate or when I toggle Virtual Super Resolution or GPU scaling it does the flickering again. I searched this up and the solution was to uninstall dragon center and that doesn't do anything, even when i uninstall everything AMD Radeon software and re install. I've tried with both 21.4.1 and 21.5.1. I then decided to just do the windows update driver AMD display and I think that downloaded the original driver and software version that the laptop originally came with. \n\nI get stuttering even when uninstalling w

In [15]:
#check if '[removed]' values are inside the columns
if '[removed]' in np.unique(amd['selftext']):
    #if there is '[removed]' values, replace those cells with '' to prepare for interaction term
    amd['selftext'] = amd['selftext'].replace('[removed]', '')

if '[removed]' in np.unique(nvidia['selftext']):
    #if there is '[removed]' values, replace those cells with '' to prepare for interaction term
    nvidia['selftext'] = nvidia['selftext'].replace('[removed]', '')

    
print(np.unique(amd['selftext']), '\n')
print(np.unique(nvidia['selftext']))

[''
 "\n\n \n\nbasically, when I download the drivers just before finishes installation it goes black screen and then i have to put the computer into sleep mode then restart. Then it says its finishes installation then starts flickering then i need to restart properly. After that, it is fine unless the computer then sleeps after not being on for some time ( its a gaming laptop) and when usually a spacebar tap should turn it on it goes black screen again. Also when i change refresh rate or when I toggle Virtual Super Resolution or GPU scaling it does the flickering again. I searched this up and the solution was to uninstall dragon center and that doesn't do anything, even when i uninstall everything AMD Radeon software and re install. I've tried with both 21.4.1 and 21.5.1. I then decided to just do the windows update driver AMD display and I think that downloaded the original driver and software version that the laptop originally came with. \n\nI get stuttering even when uninstalling w

In [16]:
#interaction term between 'title' and 'selftext' for amd dataframe
amd['title_selftext'] = amd['title'] + amd['selftext']

#drop original columns to reduce multicollinearity
amd = amd.drop(columns=['title', 'selftext'])

amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext
0,Amd,0,0,1,0,3700x vs 5600x
1,Amd,0,0,1,0,How much is the difference between Amd 7 5800 ...
2,Amd,0,0,1,0,RX 6800 vs RX 6700 XT
3,Amd,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...
4,Amd,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.


In [17]:
#interaction term between 'title' and 'selftext' for nvidia dataframe
nvidia['title_selftext'] = nvidia['title'] + nvidia['selftext']

#drop original columns to reduce multicollinearity
nvidia = nvidia.drop(columns=['title', 'selftext'])

nvidia.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext
0,nvidia,0,0,1,0,GFN Thursday: 14 New Games Join GeForce NOW
1,nvidia,0,0,1,0,Where to buy GPU in ItalyHello everyone i'm fr...
2,nvidia,0,0,1,0,Really bad performance in gta 5 onlyI hadn’t p...
3,nvidia,0,0,1,0,Unlocking gtx 1660ti I need help me with unlo...
4,nvidia,0,0,1,0,Why is my Geforce Experience doing this?


In [18]:
amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext
0,Amd,0,0,1,0,3700x vs 5600x
1,Amd,0,0,1,0,How much is the difference between Amd 7 5800 ...
2,Amd,0,0,1,0,RX 6800 vs RX 6700 XT
3,Amd,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...
4,Amd,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.


In [19]:
#remove links inside the posts
amd['title_selftext'] = amd['title_selftext'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [20]:
#remove links inside the posts
nvidia['title_selftext'] = nvidia['title_selftext'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

## Text Processing

The following are the steps taken in text processing before we begin vectorising.

1. Remove HTML using BeautifulSoup
2. Remove non-characters (punctuations etc) using RegEx
3. Change all words to lowercase
4. Tokenise the text
5. Remove stopwords
6. Lemmatise the text

In this project, due to the time constraint, we will only <b>focus on Lemmatizing</b> as it returns dictionary words.
We will not be using Stemming as the words returned may not be actual dictionary words.

Should there be more time to refine the project, Stemming should also be used and the results from both should be compared before deciding which is more suitable.

In [21]:
#get the stopwords
stopwords = stopwords.words('english')
stopwords.extend(['would', 'pc', 'new'])

stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [22]:
#Create process function to lower case, remove punctuation, tokenize, remove stopwords
def process_text(dataframe_text):
    
    #empty list for each row of text in dataframe
    processed = []
    
    #loop through each row
    for cell in dataframe_text:
        
        #instantiate lemmatizer
        lemma = WordNetLemmatizer()
    
        #remove HTML from text
        html_removed_step1 = BeautifulSoup(cell).get_text()
    
        #remove non-characters by replacing with space
        nonchar_removed_step2 = re.sub('[^a-zA-Z]', ' ', html_removed_step1)
    
        #change text to lowercase
        lower_step3 = nonchar_removed_step2.lower()
    
        #tokenise text by splitting
        token_step4 = lower_step3.split()
    
        #remove stopwords from list of text and lemmatise them
        lemma_no_stop_step5 = [lemma.lemmatize(word) for word in token_step4 if word not in stopwords]
        
        #join words for each row
        joined_step6 = (' '.join(lemma_no_stop_step5))
        
        #append processed row into list
        processed.append(joined_step6)
    
    #join the words for each post and return it
    return processed

In [23]:
#text process for amd subreddit data
amd['cleaned_texts'] = process_text(amd['title_selftext'])

In [24]:
#text process for nvidia subreddit data
nvidia['cleaned_texts'] = process_text(nvidia['title_selftext'])

In [25]:
#check if text processing caused any NULL cells
print('AMD:\n', amd.isnull().sum())
print('\nNVIDIA:\n', nvidia.isnull().sum())

AMD:
 subreddit                0
num_comments             0
num_crossposts           0
score                    0
total_awards_received    0
title_selftext           0
cleaned_texts            0
dtype: int64

NVIDIA:
 subreddit                0
num_comments             0
num_crossposts           0
score                    0
total_awards_received    0
title_selftext           0
cleaned_texts            0
dtype: int64


It seems that no NULL cells were created after text processing.

The code below is included to sieve out cells with empty strings and drop those as they will reflect as NULL cells after saving into csv.

As there will be minimal, if any, of these empty string cells created after text processing (since we are dealing with post submissions), we are comfortable with dropping them along with the NULL cells created (if any).

In [26]:
amd = amd.drop(amd[amd.cleaned_texts.isnull()].index)
amd = amd.drop(amd[amd.cleaned_texts == ''].index)
    
nvidia = nvidia.drop(nvidia[nvidia.cleaned_texts.isnull()].index)
nvidia = nvidia.drop(nvidia[nvidia.cleaned_texts == ''].index)


In [27]:
amd.shape

(11593, 7)

In [28]:
nvidia.shape

(11731, 7)

In [29]:
amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,Amd,0,0,1,0,3700x vs 5600x,x v x
1,Amd,0,0,1,0,How much is the difference between Amd 7 5800 ...,much difference amd v x worth pay x
2,Amd,0,0,1,0,RX 6800 vs RX 6700 XT,rx v rx xt
3,Amd,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...,amd wraith stealth cooler fit amd
4,Amd,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.,v xt let find


In [30]:
nvidia.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,nvidia,0,0,1,0,GFN Thursday: 14 New Games Join GeForce NOW,gfn thursday game join geforce
1,nvidia,0,0,1,0,Where to buy GPU in ItalyHello everyone i'm fr...,buy gpu italyhello everyone brazil planning tr...
2,nvidia,0,0,1,0,Really bad performance in gta 5 onlyI hadn’t p...,really bad performance gta onlyi played gta ye...
3,nvidia,0,0,1,0,Unlocking gtx 1660ti I need help me with unlo...,unlocking gtx ti need help unlocking power tem...
4,nvidia,0,0,1,0,Why is my Geforce Experience doing this?,geforce experience


In [31]:
#lowercase the 'subreddit' column
amd['subreddit'] = amd['subreddit'].apply(lambda cell: cell.lower())
nvidia['subreddit'] = nvidia['subreddit'].apply(lambda cell: cell.lower())

In [32]:
amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,amd,0,0,1,0,3700x vs 5600x,x v x
1,amd,0,0,1,0,How much is the difference between Amd 7 5800 ...,much difference amd v x worth pay x
2,amd,0,0,1,0,RX 6800 vs RX 6700 XT,rx v rx xt
3,amd,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...,amd wraith stealth cooler fit amd
4,amd,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.,v xt let find


In [33]:
nvidia.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,nvidia,0,0,1,0,GFN Thursday: 14 New Games Join GeForce NOW,gfn thursday game join geforce
1,nvidia,0,0,1,0,Where to buy GPU in ItalyHello everyone i'm fr...,buy gpu italyhello everyone brazil planning tr...
2,nvidia,0,0,1,0,Really bad performance in gta 5 onlyI hadn’t p...,really bad performance gta onlyi played gta ye...
3,nvidia,0,0,1,0,Unlocking gtx 1660ti I need help me with unlo...,unlocking gtx ti need help unlocking power tem...
4,nvidia,0,0,1,0,Why is my Geforce Experience doing this?,geforce experience


Reorder the columns so that the text columns are side-by-side

In [34]:
amd = amd[['subreddit', 'num_comments', 'num_crossposts', 'score', 'total_awards_received', 'title_selftext', 'cleaned_texts']]

In [35]:
nvidia = nvidia[['subreddit', 'num_comments', 'num_crossposts', 'score', 'total_awards_received', 'title_selftext', 'cleaned_texts']]

In [36]:
amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,amd,0,0,1,0,3700x vs 5600x,x v x
1,amd,0,0,1,0,How much is the difference between Amd 7 5800 ...,much difference amd v x worth pay x
2,amd,0,0,1,0,RX 6800 vs RX 6700 XT,rx v rx xt
3,amd,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...,amd wraith stealth cooler fit amd
4,amd,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.,v xt let find


In [37]:
nvidia.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,nvidia,0,0,1,0,GFN Thursday: 14 New Games Join GeForce NOW,gfn thursday game join geforce
1,nvidia,0,0,1,0,Where to buy GPU in ItalyHello everyone i'm fr...,buy gpu italyhello everyone brazil planning tr...
2,nvidia,0,0,1,0,Really bad performance in gta 5 onlyI hadn’t p...,really bad performance gta onlyi played gta ye...
3,nvidia,0,0,1,0,Unlocking gtx 1660ti I need help me with unlo...,unlocking gtx ti need help unlocking power tem...
4,nvidia,0,0,1,0,Why is my Geforce Experience doing this?,geforce experience


Changing 'subreddit' column values to numerical format for modelling purposes.

Values changed to:

amd = 1

nvidia = 0

In [38]:
amd['subreddit'] = amd['subreddit'].map({'amd': 1})

In [39]:
nvidia['subreddit'] = nvidia['subreddit'].map({'nvidia': 0})

In [40]:
amd.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,1,0,0,1,0,3700x vs 5600x,x v x
1,1,0,0,1,0,How much is the difference between Amd 7 5800 ...,much difference amd v x worth pay x
2,1,0,0,1,0,RX 6800 vs RX 6700 XT,rx v rx xt
3,1,1,0,1,0,Will an AMD Wraith Stealth cooler fit on an AM...,amd wraith stealth cooler fit amd
4,1,0,0,1,0,What does 1.4v do to a 3600xt? Let's find out.,v xt let find


In [41]:
nvidia.head()

,subreddit,num_comments,num_crossposts,score,total_awards_received,title_selftext,cleaned_texts
0,0,0,0,1,0,GFN Thursday: 14 New Games Join GeForce NOW,gfn thursday game join geforce
1,0,0,0,1,0,Where to buy GPU in ItalyHello everyone i'm fr...,buy gpu italyhello everyone brazil planning tr...
2,0,0,0,1,0,Really bad performance in gta 5 onlyI hadn’t p...,really bad performance gta onlyi played gta ye...
3,0,0,0,1,0,Unlocking gtx 1660ti I need help me with unlo...,unlocking gtx ti need help unlocking power tem...
4,0,0,0,1,0,Why is my Geforce Experience doing this?,geforce experience


We will drop some rows to keep the number of rows for both subreddits to 10 000 each as we want to have a balanced dataframe to work with when modelling.

In [42]:
amd = amd.head(10_000)

In [43]:
nvidia = nvidia.head(10_000)

In [44]:
amd.shape

(10000, 7)

In [45]:
nvidia.shape

(10000, 7)

## Save Cleaned Data into CSV

The cleaned data will be used in a separate Jupyter Notebook, part 3C, for EDA, Data Visualisation, Vectorisation and Modelling.

In [46]:
amd.to_csv('../data/amd_cleaned.csv', index=False)

In [47]:
nvidia.to_csv('../data/nvidia_cleaned.csv', index=False)